In [17]:
import essentia
from essentia import Pool, array
import essentia.streaming as ess
import essentia.standard as stan
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import Audio

In [18]:

# loader = stan.MonoLoader(filename="catchy-minor-melody.wav")
# audio = loader()

# plt.rcParams['figure.figsize'] = (15, 6)

# plot(pool['features.rms'][0:200])
# plt.title("rms")
# show()

# plot(pool['features.pitch'][0:200])
# plt.title("rms")
# plt.title("pitch")
# show()

# plot(pool['features.pitchConfidence'][0:200])
# plt.title("rms")
# plt.title("pitchCon")
# show()

# Audio(data=audio[0:200*2048], rate=sr)





In [19]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)

db = create_connection(r"voice-mementos-9-29-20.db")

2.6.0


In [20]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
sql_create_memos_table = """ CREATE TABLE memos (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL
                                ); """

sql_create_notes_table = """CREATE TABLE notes (
                                    id integer PRIMARY KEY,
                                    durationSeconds float NOT NULL,
                                    rms float,
                                    pitch float,
                                    pitchConfidence float,
                                    flatness float,
                                    path text,
                                    memo_id integer NOT NULL,
                                    FOREIGN KEY (memo_id) REFERENCES memos (id)
                                );"""

# create tables
if db is not None:
    # create memos table
    create_table(db, sql_create_memos_table)

    # create notes table
    create_table(db, sql_create_notes_table)
else:
    print("Error! cannot create the database connection.")

table memos already exists
table notes already exists


In [21]:
def create_memo(conn, memo):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO memos(name)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, memo)
    conn.commit()
    return cur.lastrowid

def create_note(conn, note):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO notes(durationSeconds,rms,pitch,pitchConfidence,flatness,path,memo_id)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, note)
    conn.commit()

    return cur.lastrowid

In [22]:
def get_notes_from_onsets(onsets_hfc, audio):
    # to translate, imagine 'energies' is actually your array of audio samples
    # 'onsetFrames' would be a list of sample indices corresponding to onsets.
    # 'notes' is therefore a list of arrays, each of which contains the
    # samples between a given onset and the next onset (or in case of the final
    # onset, the samples from that onset to the end of the signal)

    notes = []
    onsetFrames = onsets_hfc
    numOnsets = onsetFrames.shape[0]
    if numOnsets == 1:
        # 20150719 202811.wav only has one onset detected...
        # for now, lets skip ultra short files
        return notes;
        # notes.append(audio[onsetFrames[0]:])
    for i in range(numOnsets):
        if i == 0:
            continue

        nStart = int(sr * onsetFrames[i - 1])
        nEnd = int(sr * onsetFrames[i])

        if(nEnd-nStart) < 11025: # cull/skip notes that are shorter than 0.25 seconds
            continue

        startFrame = int(nStart/2048)
        endFrame = int(nEnd/2048)

        noteData = {}
        noteData["rms"] =             stan.Mean()(pool["features.rms"][startFrame : endFrame])
        noteData["flatness"] =        stan.Mean()(pool["features.flatnessDB"][startFrame : endFrame])
        noteData["pitch"] =           stan.Mean()(pool["features.pitch"][startFrame : endFrame])
        noteData["pitchConfidence"] = stan.Mean()(pool["features.pitchConfidence"][startFrame : endFrame])
        noteData["audio"] =           audio[nStart:nEnd]

        notes.append(noteData)

    # the idea here is assume that at the last onset marker, we create a note willy nilly to the end...
    # not important at this point. possibly dangerously presumptive
    #     if i == numOnsets - 1:
    #         notes.append(audio[int(sr * onsetFrames[i]):])

#     print(notes[5]["rms"])
#     print(notes[5]["flatness"])
#     print(notes[5]["pitch"])
#     print(notes[5]["pitchConfidence"])

#     d = []
#     for note in notes:
#         d.append(note["flatness"])

#     plot(d)

#     show()
    return notes

In [23]:
# putting it all together
from pathlib import Path
from scipy.io.wavfile import write, read
import json

def write_memo_notes_to_db(memoFile, notes):
    # maybe this name comes from the original audio file
    file_name = memoFile

    # initialize an object to store data for the clip
    memo = (file_name,)
    memo_id = create_memo(db, memo)

    # process each detected note
    plotIdx = 1
    for i, note in enumerate(notes):
        # plot the waveform
#         plot(note["audio"])
#         show()

        # choose a path to save the wav file
        file_name = file_name.replace('.', '_')
        file_name = file_name.replace(' ', '_')
        path = Path(f"{file_name}/note_{i}.wav")
        Path(f"{file_name}").mkdir(parents=True, exist_ok=True)

        durationSeconds = len(note["audio"]) / sr
        noteRecord = (
            durationSeconds,
            note["rms"],
            note["pitch"],
            note["pitchConfidence"],
            note["flatness"],
            str(path.resolve()),
            memo_id
        )

        create_note(db, noteRecord)

        # write the individual note to its own wav file
        write(path.resolve(), rate=sr, data=note["audio"])

In [24]:
# In this example we are going to look at how to perform some onset detection
# and mark them on the audio using the AudioOnsetsMarker algorithm.
#
# Onset detection consists of two main phases:
#  1- we need to compute an onset detection function, which is a function
#     describing the evolution of some parameters, which might be representative
#     of whether we might find an onset or not
#  2- performing the actual onset detection, that is given a number of these
#     detection functions, decide where in the sound there actually are onsets


sr = 44100;

import os
print(os.getcwd())

for memoFile in os.listdir(os.getcwd() + "/corpus_755"):
    audio = stan.MonoLoader(filename="corpus_755/"+memoFile)()
    loader = ess.MonoLoader(filename="corpus_755/"+memoFile)
    pool = Pool()
    
    framecutter = ess.FrameCutter(frameSize=4096, hopSize=2048, silentFrames='noise')
    windowing = ess.Windowing(type='blackmanharris62')
    c2p = ess.CartesianToPolar()
    od1 = ess.OnsetDetection(method='hfc')
    fft = ess.FFT()
    pitch = ess.PitchYinFFT(frameSize=4096)
    rms = ess.RMS()
    flatness = ess.FlatnessDB()
    spectrum = ess.Spectrum()


    # Connect streaming algorithms
    loader.audio >> framecutter.signal
    framecutter.frame >> windowing.frame >> fft.frame
    fft.fft >> c2p.complex
    c2p.phase >> od1.phase
    c2p.magnitude >> None

    windowing.frame >> spectrum.frame
    spectrum.spectrum >> od1.spectrum
    od1.onsetDetection >> (pool, 'features.hfc')

    windowing.frame >> rms.array
    rms.rms >> (pool, 'features.rms')

    spectrum.spectrum >> pitch.spectrum
    pitch.pitch >> (pool, 'features.pitch')
    pitch.pitchConfidence >> (pool, 'features.pitchConfidence')

    c2p.magnitude >> flatness.array
    flatness.flatnessDB >> (pool, 'features.flatnessDB')

    # od1.onsetDetection >> onsets.detections
    # [1] >> onsets.weights
    # onsets.onsets >> (pool, 'onsets')

    # Run streaming network
    essentia.run(loader)
    # Phase 2: compute the actual onsets locations
    onsets = stan.Onsets()

    onsets_hfc = onsets(# this algo expects a matrix, not a vector
        array([ pool['features.hfc'] ]),

        # you need to specify weights, but as there is only a single
        # function, it doesn't actually matter which weight you give it
        [ 1 ])
    notes = get_notes_from_onsets(onsets_hfc, audio)
    write_memo_notes_to_db(memoFile, notes)
    print('👀 ' + memoFile + " : " + str(len(notes)) + " notes")



/notebooks
👀 20160316 011636.wav : 75 notes
👀 20160318 163105.wav : 13 notes
👀 20160322 074850.wav : 6 notes
👀 20160330 224050.wav : 8 notes
👀 20160331 144333.wav : 142 notes
👀 20160331 164940.wav : 19 notes
👀 20160331 172724.wav : 28 notes
👀 20160401 010721.wav : 2 notes
👀 20160401 010818.wav : 5 notes
👀 20160401 012315.wav : 10 notes
👀 20160402 005657.wav : 11 notes
👀 20160403 194352.wav : 35 notes
👀 20160404 205457.wav : 41 notes
👀 20160410 170314.wav : 12 notes
👀 20160410 230055.wav : 0 notes
👀 20160411 012114.wav : 11 notes
👀 20160411 222612.wav : 88 notes
👀 20160411 223056.wav : 26 notes
👀 20160411 223336.wav : 54 notes
👀 20160411 223606.wav : 16 notes
👀 20160411 223918.wav : 16 notes
👀 20160411 233538.wav : 0 notes
👀 20160412 101440.wav : 55 notes
👀 20160414 000052.wav : 48 notes
👀 20160414 184018.wav : 75 notes
👀 20160417 162641.wav : 86 notes
👀 20160417 184636.wav : 65 notes
👀 20160423 104722.wav : 34 notes
👀 20160423 110928.wav : 50 notes
👀 20160423 111556.wav : 72 notes
👀 20

👀 20150719 003126.wav : 14 notes
👀 20150719 015947.wav : 20 notes
👀 20150719 162924.wav : 1 notes
👀 20150719 202811.wav : 0 notes
👀 20150721 194704.wav : 5 notes
👀 20150722 195847.wav : 3 notes
👀 20150722 220426.wav : 20 notes
👀 20150724 140531.wav : 4 notes
👀 20150726 123901.wav : 59 notes
👀 20150726 124431.wav : 31 notes
👀 20150727 214016.wav : 71 notes
👀 20150731 192825.wav : 131 notes
👀 20150801 011520.wav : 159 notes
👀 20150801 172851.wav : 0 notes
👀 20150802 141907.wav : 0 notes
👀 20150803 205313.wav : 62 notes
👀 20150803 221412.wav : 32 notes
👀 20150804 090124.wav : 35 notes
👀 20150804 201312.wav : 14 notes
👀 20150804 222218.wav : 23 notes
👀 20150805 191347.wav : 1 notes
👀 20150807 225630.wav : 3 notes
👀 20150808 182015.wav : 0 notes
👀 20150810 122951.wav : 5 notes
👀 20150810 185355.wav : 8 notes
👀 20150810 192508.wav : 30 notes
👀 20150814 135649.wav : 42 notes
👀 20150814 142042.wav : 2 notes
👀 20150816 194927.wav : 27 notes
👀 20150818 203025.wav : 5 notes
👀 20150818 220558.wav 

👀 20140601 161536.wav : 12 notes
👀 20140603 162426.wav : 15 notes
👀 20140603 165329.wav : 4 notes
👀 20140603 205541.wav : 4 notes
👀 20140607 204435.wav : 163 notes
👀 20140607 210845.wav : 88 notes
👀 20140607 211509.wav : 41 notes
👀 20140608 185705.wav : 62 notes
👀 20140608 190757.wav : 55 notes
👀 20140608 192854.wav : 51 notes
👀 20140608 193209.wav : 64 notes
👀 20140608 193733.wav : 32 notes
👀 20140608 194134.wav : 60 notes
👀 20140608 230620.wav : 7 notes
👀 20140608 234418.wav : 20 notes
👀 20140610 222800.wav : 27 notes
👀 20140616 103803.wav : 108 notes
👀 20140617 210603.wav : 7 notes
👀 20140617 221627.wav : 14 notes
👀 20140617 223222.wav : 5 notes
👀 20140625 200858.wav : 16 notes
👀 20140626 224002.wav : 59 notes
👀 20140626 231207.wav : 2 notes
👀 20140626 234240.wav : 1 notes
👀 20140629 005751.wav : 87 notes
👀 20140629 195114.wav : 4 notes
👀 20140629 231755.wav : 9 notes
👀 20140630 123246.wav : 5 notes
👀 20140707 160926.wav : 8 notes
👀 20140708 194423.wav : 31 notes
👀 20140711 002238.w

RuntimeError: In MonoLoader.compute: std::bad_alloc